In [1]:
import pandas as pd
import datetime as dt
import numpy as np


In [2]:
excel_file = pd.ExcelFile("queues_2023_clean_data.xlsx")
interconnection_data = excel_file.parse("data")
print(interconnection_data.head())

      q_id   q_status      q_date  q_year ia_date wd_date on_date entity  \
0   GIA-50  withdrawn  12/18/2014  2014.0     NaN     NaN     NaN    AEC   
1  GIA-105     active   7/21/2022  2022.0     NaN     NaN     NaN    AEC   
2   GIA-83     active   6/19/2019  2019.0     NaN     NaN     NaN    AEC   
3  GIA-113     active   1/13/2023  2023.0     NaN     NaN     NaN    AEC   
4   GIA-73  withdrawn  11/19/2018  2018.0     NaN     NaN     NaN    AEC   

  project_name developer  ... prop_year IA_status_raw      IA_status_clean  \
0          NaN       NaN  ...    2016.0     Withdrawn            Withdrawn   
1          NaN       NaN  ...    2025.0            FS     Facilities Study   
2          NaN       NaN  ...    2025.0           GIA          IA Executed   
3          NaN       NaN  ...    2026.0           SIS  System Impact Study   
4          NaN       NaN  ...    2021.0     Withdrawn            Withdrawn   

  type_clean  type1 type2 type3     mw1 mw2  mw3  
0       Wind   Wind   N

In [4]:
def categorical_change(data,column):
    """Changes a column into categorical variables"""
    i=0
    seen = {}
    categorical = []
    for _, row in data.iterrows():
        item = row[column]
        if item not in seen:
            seen[item] = i
            i+=1
        categorical.append(seen[item])
    data[column] = categorical

date_columns = ["q_date","ia_date","wd_date","on_date"]
for col in date_columns:
    interconnection_data[col] = pd.to_datetime(interconnection_data[col],errors = "coerce")
print(interconnection_data.head())

      q_id   q_status     q_date  q_year ia_date wd_date on_date entity  \
0   GIA-50  withdrawn 2014-12-18  2014.0     NaT     NaT     NaT    AEC   
1  GIA-105     active 2022-07-21  2022.0     NaT     NaT     NaT    AEC   
2   GIA-83     active 2019-06-19  2019.0     NaT     NaT     NaT    AEC   
3  GIA-113     active 2023-01-13  2023.0     NaT     NaT     NaT    AEC   
4   GIA-73  withdrawn 2018-11-19  2018.0     NaT     NaT     NaT    AEC   

  project_name developer  ... prop_year IA_status_raw      IA_status_clean  \
0          NaN       NaN  ...    2016.0     Withdrawn            Withdrawn   
1          NaN       NaN  ...    2025.0            FS     Facilities Study   
2          NaN       NaN  ...    2025.0           GIA          IA Executed   
3          NaN       NaN  ...    2026.0           SIS  System Impact Study   
4          NaN       NaN  ...    2021.0     Withdrawn            Withdrawn   

  type_clean  type1 type2 type3     mw1 mw2  mw3  
0       Wind   Wind   NaN   N